In [1]:
%reload_ext autoreload
%autoreload 2

In [13]:
%%writefile tmp.bpf.c
struct task_struct {
  int i;
  int j;
  int k;
  struct inner {
    int a;
    int b;
  } s;
} __attribute__((preserve_access_index));

int f(struct task_struct* foo) { 
    return foo->s.b;
}

Overwriting tmp.bpf.c


In [12]:
!clang -g -O2 -target bpf -c tmp.bpf.c -o tmp.bpf.o
!llvm-objdump-18 --disassemble --reloc --source tmp.bpf.o


tmp.bpf.o:	file format elf64-bpf

Disassembly of section kprobe/f:

0000000000000000 <f>:
;     return (int) foo->s.b;
       0:	61 10 10 00 00 00 00 00	r0 = *(u32 *)(r1 + 0x10)
		0000000000000000:  CO-RE <byte_off> [2] struct task_struct::s.b (0:3:1)
       1:	95 00 00 00 00 00 00 00	exit


In [82]:
from depsurf import gen_min_btf, dump_btf

from pathlib import Path

f = gen_min_btf(Path('tmp.bpf.o'), overwrite=True)
dump_btf(f, overwrite=True)

[system.py:19] INFO: Running command: "/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/bpftool/src/bpftool -d gen min_core_btf /sys/kernel/btf/vmlinux tmp.bpf.min.btf tmp.bpf.o"
libbpf: CO-RE relocating [2] struct task_struct: found target candidate [2] struct task_struct in [vmlinux]
libbpf: prog 'kprobe/f': relo #0: <byte_off> [2] struct task_struct.k (0:3:0 @ offset 12)
libbpf: prog 'kprobe/f': relo #0: matching candidate #0 <byte_off> [2] struct task_struct.k (0:3:0 @ offset 12)
[system.py:19] INFO: Running command: "/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/bpftool/src/bpftool btf dump file tmp.bpf.min.btf format c > tmp.bpf.min.h"
[system.py:19] INFO: Running command: "/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/bpftool/src/bpftool btf dump file tmp.bpf.min.btf format raw > tmp.bpf.min.txt"
[system.py:19] INFO: Running command: "/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/bpftool/src/bpftool btf dump file tmp.bpf.min.btf --json > tmp.bpf.min.json"


In [3]:
from depsurf import BPFObjectFile, BCC_OUTPUT_PATH


result = {}
for path in sorted(BCC_OUTPUT_PATH.glob("*.bpf.o")):
    bpfobj = BPFObjectFile(path)
    bpfobj.dump_btf(overwrite=False)
    result[path.stem] = bpfobj.get_relo().get_deps()
    # print(path.stem, bpfobj.get_relo().get_deps())

for name, deps in result.items():
    print(name)
    for dep in deps:
        print("  ", dep)
    print()

# bpfobj = BPFObjectFile("tmp.bpf.o")
# bpfobj = BPFObjectFile(BCC_OUTPUT_PATH / "tcptracer.bpf.o")
# bpfobj.objdump()
# bpfobj.get_relo()

[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.h
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.txt
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.json
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.h
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.txt
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.json
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/biolatency.bpf.h
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/biolatency.bpf.txt
[bpftool.py:49] INFO: Using /Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/biolatency.bpf.json
[bpftool.py:49] INFO: Using /Users/szhong/Down

bashreadline.bpf
   struct user_pt_regs::regs

bindsnoop.bpf
   struct in6_addr::in6_u
   struct in6_addr::u6_addr32
   struct inet_sock::bind_address_no_port
   struct inet_sock::freebind
   struct inet_sock::inet_flags
   struct inet_sock::inet_saddr
   struct inet_sock::inet_sport
   struct inet_sock::transparent
   struct sock::__sk_common
   struct sock::sk_protocol
   struct sock_common::skc_bound_dev_if
   struct sock_common::skc_reuse
   struct sock_common::skc_reuseport
   struct sock_common::skc_v6_rcv_saddr
   struct socket::sk
   struct user_pt_regs::regs

biolatency.bpf
   struct gendisk::first_minor
   struct gendisk::major
   struct request::cmd_flags
   struct request::q
   struct request::rq_disk
   struct request_queue::disk
   struct request_queue::elevator

biopattern.bpf
   struct trace_event_raw_block_rq_complete::dev
   struct trace_event_raw_block_rq_complete::nr_sector
   struct trace_event_raw_block_rq_complete::sector
   struct trace_event_raw_block_rq_comple